In [2]:
# import library
import requests
from bs4 import BeautifulSoup
import unicodedata
import mysql.connector
import ipynb
from ipynb.fs.full.SqlQuery import brandAdd
from ipynb.fs.full.SqlQuery import categorieAdd
from ipynb.fs.full.SqlQuery import AddProduct
from ipynb.fs.full.SqlQuery import getCagId
from ipynb.fs.full.SqlQuery import getBrandId
from ipynb.fs.full.SqlQuery import AddProductAttribut
from ipynb.fs.full.SqlQuery import AddLocalStoreProduct
from ipynb.fs.full.SqlQuery import Addlocalstors

In [5]:
# get All wiki category links
def wikiGetLinks():
    list=[]
    web = requests.get('https://www.wiki.tn/')
    parse = BeautifulSoup(web.content, 'html.parser')
    catalogue= parse.find_all('li', class_='parent dropdown-submenu')
    for cag in catalogue:
        link=cag.a['href'].replace('é','e')
        info=cag.a.span.text.strip()
        dect={
            'info':info,
            'link':link
            }
        list.append(dect)
    return list

# MysQl Data Base connection
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="scraping_data"
)

In [6]:
# extract products number from string
def getTotalProduits(ch):
    s=''
    t=''
    final=''
    i=0
    ch=ch.replace(' ','')
    for c in ch:
        s=s+c
        i=i+1
        if(c == '-'):
            final=ch[i+3:len(ch)]

    for f in final:
        if(f.isdigit()):
            t=t+f
    return int(t)

In [16]:
#print(getTotalProduits('Résultats 1 - 12 sur 435 produits'))

435


In [9]:
#exract wiki data from details page
def wiki_details(url):
    list=[]
    web = requests.get(url)
    parse = BeautifulSoup(web.content, 'html.parser')
    try:
        brand_logo= parse.find('span', class_='marque').img['src']
    except:
        brand_logo=''
    try:
        sku=parse.find('p', id='product_reference').span.text.strip()
    except:
        sku='undfined'     
    try:
        des= parse.find('div', id='short_description_block').text.strip()
    except:
        des='None'
    try:
        fich_tech= parse.find('table', class_='table-data-sheet').find_all('tr')
        for f in fich_tech:
            title=f.td.text
            feautre= [t.text for t in f.find_all('td')][1]
            fich_dec={
                'label':title,
                'data':feautre
            }
            list.append(fich_dec)
    except:
        list=[]
    article={
        'sku':sku,
        'brand_logo':brand_logo,
        'des':des,
        'fiche_technique':list
    }
    return article

In [10]:
#convert string price to integer
def priceConv(price):
    c=''
    for p in price:
        if(p.isdigit()):
            c=c+p
    return int(c)

In [18]:
#wiki main scraping algorithem
i=0
list= wikiGetLinks()
mycursor = mydb.cursor()
for l in list:
    info=l['info']
    link=l['link']
    categorieAdd(info, link, 18)
    supCatId=getCagId(info,18)
    web = requests.get(link)
    parse = BeautifulSoup(web.content, 'html.parser')
    art= parse.find_all('div', class_='product-container product-block')
    try:
        nb= parse.find('div',class_='product-count pull-right').text
    except:
        nb='Résultats 1 - 12 sur 5 produits'
#----------get the orginal page
    i=0
    for x in range(1,30):
        print(f'{info}=>{getTotalProduits(nb)}=>{link}')
        if(x==1):
            link =link+'#/page-'+str(x)
        else:
            link=link[:-1]
            if(x>10):
                link=link[:-1]
            link=link+str(x)
        text = requests.get(link)
        parse = BeautifulSoup(text.content, 'html.parser')
        if(i < getTotalProduits(nb)):
            for a in art:
                i=i+1
                name= a.find('a', class_='product-name').text
                try:
                    price = a.find('span', class_='price product-price').text
                    price = priceConv(price)
                except:
                    price=0
                art_link=a.find('a', class_='product-name').get('href')
                try:
                    brand= a.find('div', class_='logo_marque').img['alt']
                except:
                    brand='Unkown'
                img = a.find('a', class_='product_img_link').img['src']
                stock= a.find('span', class_='availability').span.text
                det= wiki_details(art_link)
                brandAdd(brand, det['brand_logo'], 18)
                supBrandId=getBrandId(brand, 18)
                prodId=AddProduct(name, art_link, det['sku'], det['des'], price, stock, 18, supBrandId, supCatId, img)
                if(det['fiche_technique']):
                    AddProductAttribut(det['fiche_technique'], prodId)
        else:
            break

Data Base Categorie Duplication Error

Smartphone & Mobile=>226=>https://www.wiki.tn/c/smartphone-mobile-258.html
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Smartphone & Mobile=>226=>https://www.wiki.tn/c/smartphone-mobile-258.html#/page-1
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Smartphone & Mobile=>226=>https://www.wiki.tn/c/smartphone-mobile-258.html#/page-2
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error
Product Duplicate Error


KeyboardInterrupt: 